In [3]:
__author__ = 'himanshu'
import hashlib
from sqlalchemy import create_engine, Table, ForeignKey, Enum
from datetime import date, datetime
from sqlalchemy.orm import sessionmaker, relationship, backref, scoped_session
from sqlalchemy import Column, Integer, Boolean, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.pool import SingletonThreadPool
from sqlalchemy.ext.hybrid import hybrid_property
import os
Base = declarative_base()
from sqlalchemy.sql.expression import ColumnElement
from sqlalchemy.ext.compiler import compiles
from sqlalchemy import inspect

In [4]:
class User(Base):
     __tablename__ = 'user'

     id = Column(Integer, primary_key=True)
     fullname = Column(String)
     osf_login = Column(String)
     osf_password = Column(String)
     osf_path = Column(String)
     oauth_token = Column(String)
     osf_id = Column(String)

     #todo: enforce category = PROJECT condition for projects
     nodes = relationship(
         "Node",
         backref =backref('user'),
         cascade="all, delete-orphan"
     )

     files = relationship(
         "File",
         backref=backref('user'),
         cascade="all, delete-orphan"
     )

     @hybrid_property
     def projects(self):
         projects =[]
         for node in self.nodes:
             if node.category == Node.PROJECT:
                 projects.append(node)
         return projects

     def __repr__(self):
       return "<User(fullname={}, osf_password={}, osf_path={})>".format(
                             self.fullname, self.osf_password, self.osf_path)

In [5]:
class PathColumn (ColumnElement):
    def __init__(self, entity):
        insp = inspect(entity)
        self.entity = insp.selectable
 
@compiles(PathColumn)
def compile_path_column(element, compiler, **kwargs):
    return "%s.path" % compiler.process(element.entity, ashint=True)

class Node(Base):
    __tablename__ = "node"

    PROJECT='project'
    COMPONENT='component'

    id = Column(Integer, primary_key=True)
    title = Column(String)
    path = Column(String)
    hash = Column(String)
    category = Column(Enum(PROJECT, COMPONENT))
    date_modified = Column(DateTime)
    osf_id = Column(String)

    deleted = Column(Boolean, default=False)

    user_id = Column(Integer, ForeignKey('user.id'))
    parent_id = Column(Integer, ForeignKey('node.id'))
    components = relationship(
        "Node",
        backref=backref('parent', remote_side=[id]),
        # cascade="all, delete-orphan" #todo: watchdog crawls up so cascade makes things fail on recursive delete. may want to have delete just ignore fails.
    )
    files = relationship(
        "File",
        backref=backref('node')
    )

    @hybrid_property
    def path(self):
        """Recursively walk up the path of the node. Top level node joins with the osf folder path of the user
        """
        # +os.path.sep+ instead of os.path.join: http://stackoverflow.com/a/14504695
        if self.parent:
            return os.path.join(self.parent.path, self.title)
        else:
            return os.path.join(self.user.osf_path , self.title)

    @path.expression
    def path(cls):
        return PathColumn(cls)

    def update_hash(self, blocksize=2**20):
        pass
        #todo: what to do in this case?

    def update_time(self, dt=None):
        if dt:
            self.date_modified = dt
        else:
            self.date_modified = datetime.now()

    def __repr__(self):
        return "<Node ({}), category={}, title={}, path={}, parent_id={}>".format(
            self.id, self.category, self.title, self.path, self.parent_id
        )

In [6]:
class File(Base):
    __tablename__ = "file"

    FOLDER ='folder'
    FILE='file'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    path = Column(String)
    # guid = Column(String)
    hash = Column(String)
    type = Column(Enum(FOLDER,FILE))
    date_modified = Column(DateTime)
    osf_id = Column(String)

    deleted = Column(Boolean)

    user_id = Column(Integer, ForeignKey('user.id'))
    node_id = Column(Integer, ForeignKey('node.id'))
    parent_id = Column(Integer, ForeignKey('file.id'))
    files = relationship(
        "File",
        backref=backref('parent', remote_side=[id]),
        # cascade="all, delete-orphan",  #todo: watchdog crawls up so cascade makes things fail on recursive delete. may want to have delete just ignore fails.
    )

    @hybrid_property
    def path(self):
        """Recursively walk up the path of the file/folder. Top level file/folder joins with the path of the containing node.
        """
        # +os.path.sep+ instead of os.path.join: http://stackoverflow.com/a/14504695
        if self.parent:
            return os.path.join(self.parent.path, self.name)
        else:
            return os.path.join(self.node.path ,self.name)

    def update_hash(self, blocksize=2**20):
        m = hashlib.md5()
        if self.type == File.FILE:
            with open(self.path,"rb") as f:
                while True:
                    buf = f.read(blocksize)
                    if not buf:
                        break
                    m.update(buf)
        else:
            pass
            #todo: what to do in this case?
            # m.update()
        self.hash = m.hexdigest()

    def update_time(self, dt=None):
        if dt:
            self.date_modified = dt
        else:
            self.date_modified = datetime.now()

    def __repr__(self):
        return "<File ({}), type={}, name={}, path={}, parent_id={}>".format(
            self.id, self.type, self.name, self.path, self.parent
        )

In [7]:
engine = create_engine('sqlite:///:memory:', echo=False)
Base.metadata.create_all(engine)
session_factory = sessionmaker(bind=engine)
Session = scoped_session(session_factory)
session = Session()

In [8]:
#USER
 # id = Column(Integer, primary_key=True)
 #     fullname = Column(String)
 #     osf_login = Column(String)
 #     osf_password = Column(String)
 #     osf_path = Column(String)
 #     oauth_token = Column(String)
 #     osf_id = Column(String)

user = User(fullname="Himanshu Ojha", osf_id="himan", osf_path="/home/himanshu/OSF-Offline/dumbdir/")
session.add(user)
session.commit()

In [9]:
#NODE
# id = Column(Integer, primary_key=True)
#     title = Column(String)
#     path = Column(String)
#     hash = Column(String)
#     category = Column(Enum(PROJECT, COMPONENT))
#     date_modified = Column(DateTime)
#     osf_id = Column(String)
n1 = Node(title='my project', category=Node.PROJECT)
session.add(n1)
session.commit()

In [10]:
user.nodes.append(n1)

In [11]:
print(user.nodes)
print(user.projects)
print(user.files)

[<Node (1), category=project, title=my project, path=/home/himanshu/OSF-Offline/dumbdir/my project, parent_id=None>]
[<Node (1), category=project, title=my project, path=/home/himanshu/OSF-Offline/dumbdir/my project, parent_id=None>]
[]


In [12]:
n1.components.append(Node(title='my component', category=Node.COMPONENT))
n3 = Node(title='my other component', category=Node.COMPONENT)
n1.components.append(n3)
session.add(n1)
session.commit()

In [13]:
print(user.projects)
print(user.nodes)
print(user.files)

print(n1.components)
print(n1.files)
print(n1.user)

[<Node (1), category=project, title=my project, path=/home/himanshu/OSF-Offline/dumbdir/my project, parent_id=None>]
[<Node (1), category=project, title=my project, path=/home/himanshu/OSF-Offline/dumbdir/my project, parent_id=None>]
[]
[<Node (2), category=component, title=my component, path=/home/himanshu/OSF-Offline/dumbdir/my project/my component, parent_id=1>, <Node (3), category=component, title=my other component, path=/home/himanshu/OSF-Offline/dumbdir/my project/my other component, parent_id=1>]
[]
<User(fullname=Himanshu Ojha, osf_password=None, osf_path=/home/himanshu/OSF-Offline/dumbdir/)>


In [14]:
session.add(user)
session.commit()

In [15]:
test2node = Node(title='my project', category=Node.PROJECT, user=user)
test3node = Node(title='mycompoent', category=Node.COMPONENT, user=user, parent= test2node)
session.add(test2node)
session.commit()

In [16]:
test3node.parent = None
session.add(user)
session.commit()
print(test3node)

<Node (5), category=component, title=mycompoent, path=/home/himanshu/OSF-Offline/dumbdir/mycompoent, parent_id=None>


In [17]:
session.query(Node).filter(Node.path == '/home/himanshu/OSF-Offline/dumbdir/mycompoent').all()

OperationalError: (sqlite3.OperationalError) no such column: node.path [SQL: 'SELECT node.id AS node_id, node.title AS node_title, node.hash AS node_hash, node.category AS node_category, node.date_modified AS node_date_modified, node.osf_id AS node_osf_id, node.deleted AS node_deleted, node.user_id AS node_user_id, node.parent_id AS node_parent_id \nFROM node \nWHERE node.path = ?'] [parameters: ('/home/himanshu/OSF-Offline/dumbdir/mycompoent',)]